# Diabetes Hospitalizations

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 120)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

### Location of Dataset:

https://data.world/uci/diabetes-130-us-hospitals-for-years-1999-2008

### Article for which data was collected originally:
https://www.hindawi.com/journals/bmri/2014/781670/

## Import Data and Cleaning

In [2]:
df = pd.read_csv('https://query.data.world/s/fzhdybgova7pqh6amwfzrnhumdc26t')

Need to remove time-series elements from when the same patient visited several times. Kept the first entry and removed all others.

In [3]:
df.drop_duplicates(subset='patient_nbr', inplace=True) # 101,766 to 71,518 observations

Weight, Payer Code, and Medical Specialty were missing >50% of values.

In [4]:
df.drop(['encounter_id','patient_nbr','weight', 'payer_code', 'medical_specialty'], axis=1, inplace=True)

Dropping missing values from Race column.

In [5]:
df = df[df.race != '?'] # about 1,000 obs
df = df[df.gender != 'Unknown/Invalid'] # 1 obs

Changed target variable to 0-2 for classes (No readmit, less than 30 days, more than 30 days).

In [6]:
df.readmitted.replace({'NO': 0, '<30': 1, '>30': 2}, inplace=True)

Parsing the ICD-9 medical billing codes, in order to grab just 250.xx coding diabetes.

In [7]:
df = df[pd.to_numeric(df['diag_1'], errors='coerce').notnull()] # Select non-null values after changing values to
df = df[pd.to_numeric(df['diag_2'], errors='coerce').notnull()] # numeric (removing V27/V57 entries and ?)
df = df[pd.to_numeric(df['diag_3'], errors='coerce').notnull()] #

df.diag_1 = df.diag_1.astype('float64') # Can now recast from Object to Float64 dtypes (want float to preserve
df.diag_2 = df.diag_2.astype('float64') # ICD-9 decimals as part of the billing code)
df.diag_3 = df.diag_3.astype('float64') #

In [8]:
df.shape

(62754, 45)

## EDA

In [ ]:
df.readmitted.value_counts().plot(kind='bar')

In [ ]:
df.age.value_counts().plot(kind='bar')

In [ ]:
df.time_in_hospital.plot(kind='hist')

## Feature Engineering

Was the patient given a A1C test at all?

In [9]:
df['A1C_test'] = np.where(df.A1Cresult == 'None', 0, 1)

Was the patient's meds changed during the hospitalization?

In [10]:
df.change = np.where(df.change == 'No', 0, 1)

Was the patient tested and meds changed?

In [11]:
df['A1C_test_and_changed'] = np.where((df.change == 1) & (df.A1C_test == 1), 1, 0)

From domain knowledge, patients are readmitted at different rates based on their age brackets.

In [12]:
conditions = [
    (df.age ==  '[0-10)') | (df.age == '[10-20)') | (df.age == '[20-30)'),
    (df.age == '[30-40)') | (df.age == '[40-50)') | (df.age == '[50-60)'),
    (df.age == '[60-70)') | (df.age == '[70-80)') | (df.age == '[80-90)') | (df.age == '[90-100')]

choices = [
    '[0-30)',
    '[30-60]',
    '[60-100)']

In [13]:
df['binned_age'] = np.select(conditions, choices, default=np.nan)

In [14]:
df = df[df.binned_age != 'nan']

In [15]:
df.drop(['age'], axis=1, inplace=True) # Dropping for correlation

Is diabetes one of the three primary diagnoses for the hospitalization?

In [16]:
df['diabetes_as_diag_1'] = np.where((df.diag_1 >= 250) & (df.diag_1 <251), 1, 0)
df['diabetes_as_diag_2'] = np.where((df.diag_2 >= 250) & (df.diag_2 <251), 1, 0)
df['diabetes_as_diag_3'] = np.where((df.diag_3 >= 250) & (df.diag_3 <251), 1, 0)

In [17]:
df.drop(['diag_1', 'diag_2', 'diag_3'], axis=1, inplace=True) # Dropping for correlation

In [18]:
meds_to_remove = ['repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'tolbutamide', 
            'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'examide', 
            'citoglipton', 'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone', 
            'metformin-rosiglitazone', 'metformin-pioglitazone']
df.drop(meds_to_remove, axis=1, inplace=True)

In [19]:
df.shape

(61025, 28)

In [20]:
df

,race,gender,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,max_glu_serum,A1Cresult,metformin,glipizide,glyburide,insulin,change,diabetesMed,readmitted,A1C_test,A1C_test_and_changed,binned_age,diabetes_as_diag_1,diabetes_as_diag_2,diabetes_as_diag_3
1,Caucasian,Female,1,1,7,3,59,0,18,0,0,0,9,None,None,No,No,No,Up,1,Yes,2,0,0,[0-30),0,1,0
3,Caucasian,Male,1,1,7,2,44,1,16,0,0,0,7,None,None,No,No,No,Up,1,Yes,0,0,0,[30-60],0,1,0
4,Caucasian,Male,1,1,7,1,51,0,8,0,0,0,5,None,None,No,Steady,No,Steady,1,Yes,0,0,0,[30-60],0,0,1
5,Caucasian,Male,2,1,2,3,31,6,16,0,0,0,9,None,None,No,No,No,Steady,0,Yes,2,0,0,[30-60],0,0,1
7,Caucasian,Male,1,1,7,5,73,0,12,0,0,0,8,None,None,No,No,Steady,No,0,Yes,2,0,0,[60-100),0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101754,Caucasian,Female,1,1,7,9,50,2,33,0,0,0,9,None,>7,No,No,Up,Steady,1,Yes,2,1,1,[60-100),0,0,1
101755,Other,Female,1,1,7,14,73,6,26,0,1,0,9,None,>8,No,Steady,No,Up,1,Yes,2,1,1,[30-60],0,0,0
101756,Other,Female,1,1,7,2,46,6,17,1,1,1,9,None,None,No,No,No,Steady,0,Yes,2,0,0,[60-100),0,0,0
101758,Caucasian,Female,1,1,7,5,76,1,22,0,1,0,9,None,None,No,No,No,Up,1,Yes,0,0,0,[60-100),0,0,0


## Feature Selection

In [21]:
X = df.drop('readmitted', axis = 1)
y = df.readmitted

### Generating Polynomical/Interaction Features

In [22]:
continuous_variables = ['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 
                       'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']
categorical_variables = [x for x in X.columns if x not in continuous_variables]

In [23]:
df_cont = X[continuous_variables]
df_cate = X[categorical_variables]

Generating polynomial and interaction features for continuous variables.

In [24]:
poly_cont = PolynomialFeatures(2)
poly_df_cont_data = poly_cont.fit_transform(df_cont)
poly_df_cont_cols = poly_cont.get_feature_names(df_cont.columns)
poly_df_cont = pd.DataFrame(poly_df_cont_data, columns=poly_df_cont_cols) # 45 features

In [25]:
poly_df_cont.shape

(61025, 45)

Generating interaction features for categorical variables.

In [26]:
dummy_df_cate = pd.get_dummies(df_cate)

In [27]:
poly_cate = PolynomialFeatures(interaction_only=True)
poly_df_cate_data = poly_cate.fit_transform(dummy_df_cate)
poly_df_cate_cols = poly_cate.get_feature_names(dummy_df_cate.columns)
poly_df_cate = pd.DataFrame(poly_df_cate_data, columns=poly_df_cate_cols) # 1,036 features

In [28]:
poly_df_cate.shape

(61025, 1036)

Concatenating our cont and cate features back together.

In [29]:
df_all = pd.concat([poly_df_cont, poly_df_cate], axis = 1) # 1,081 features total

In [30]:
df_all.shape

(61025, 1081)

### Train Test Split

In [31]:
X_train, X_test, y_train, y_test = train_test_split(df_all, y, random_state=30)
print("Training set - Features: ", X_train.shape, "Target: ", y_train.shape)
print("Training set - Features: ", X_test.shape, "Target: ",y_test.shape)

Training set - Features:  (45768, 1081) Target:  (45768,)
Training set - Features:  (15257, 1081) Target:  (15257,)


Scaling our data: fit to the training set then apply to both train and test sets.

In [32]:
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns = X_train.columns)

### Removing Correlated Features

In [33]:
corr_matrix = X_train.corr().abs()

In [34]:
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

In [35]:
correlated = [column for column in upper.columns if any(upper[column] > 0.75)]

X_train.drop(columns=correlated, inplace=True)
X_test.drop(columns=correlated, inplace=True)

## Modeling

In [43]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

def print_metrics(labels, preds):
    print("Precision Score: {}".format(precision_score(labels, preds, average='weighted')))
    print("Recall Score: {}".format(recall_score(labels, preds, average='weighted')))
    print("Accuracy Score: {}".format(accuracy_score(labels, preds)))
    print("F1 Score: {}".format(f1_score(labels, preds, average='weighted')))

In [37]:
from sklearn.linear_model import LogisticRegression

In [40]:
logreg = LogisticRegression(random_state=30, max_iter=1000)
model_log = logreg.fit(X_train, y_train)

In [41]:
y_pred = model_log.predict(X_test)

In [44]:
print_metrics(y_pred, y_test)

Precision Score: 0.8586587197631513
Recall Score: 0.6009700465360163
Accuracy Score: 0.6009700465360163
F1 Score: 0.6909528110282128
